In [1]:
import argparse
import os
import logging
from numpy import ogrid
import torch
from datasets import *
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
import networkx as nx
from torch_geometric.utils import dense_to_sparse
from torch_geometric.utils.convert import to_networkx
import pickle
from torch_geometric.data import InMemoryDataset

from rewire_functions import (
    rewire_Graph,
    rewire_Graph_local_bridges,
    rewire_Graph_betweenness,
    apply_rewiring_strategy)

In [13]:
data = torch.load("/teamspace/studios/this_studio/gnn-comparison/DATA/CHEMICAL/PROTEINS_rewired&originali_preprocessed.pt")

In [14]:
data[0]

Data(edge_index=[2, 162], x=[42, 3], y=[1], rewired_edge_index=[2, 81])

In [12]:


num_graphs = slices['x'].size(0) - 1  # assuming 'x' is present in all graphs

for i in range(num_graphs):
    graph = Data()
    for key in data.keys:
        start, end = slices[key][i].item(), slices[key][i+1].item()
        item = data[key][start:end] if data[key].dim() > 0 else data[key]
        graph[key] = item
    
    if 'rewired_edge_index' in graph:
        print(f"Graph {i} has rewired_edge_index with shape: {graph.rewired_edge_index.shape}")
    else:
        print(f"Graph {i} is missing rewired_edge_index")



Graph 0 is missing rewired_edge_index
Graph 1 is missing rewired_edge_index
Graph 2 is missing rewired_edge_index
Graph 3 is missing rewired_edge_index
Graph 4 is missing rewired_edge_index
Graph 5 is missing rewired_edge_index
Graph 6 is missing rewired_edge_index
Graph 7 is missing rewired_edge_index
Graph 8 is missing rewired_edge_index
Graph 9 is missing rewired_edge_index
Graph 10 is missing rewired_edge_index
Graph 11 is missing rewired_edge_index
Graph 12 is missing rewired_edge_index
Graph 13 is missing rewired_edge_index
Graph 14 is missing rewired_edge_index
Graph 15 is missing rewired_edge_index
Graph 16 is missing rewired_edge_index
Graph 17 is missing rewired_edge_index
Graph 18 is missing rewired_edge_index
Graph 19 is missing rewired_edge_index
Graph 20 is missing rewired_edge_index
Graph 21 is missing rewired_edge_index
Graph 22 is missing rewired_edge_index
Graph 23 is missing rewired_edge_index
Graph 24 is missing rewired_edge_index
Graph 25 is missing rewired_edge_in

In [6]:
import json

with open("/teamspace/studios/this_studio/gnn-comparison/DATA/PROTEINS_full/processed/PROTEINS_full_splits.json") as f:
    splits = json.load(f)

# Print one entry to understand structure
print(json.dumps(splits[0], indent=2))


{
  "test": [
    6,
    18,
    28,
    29,
    35,
    43,
    51,
    59,
    63,
    80,
    88,
    93,
    100,
    101,
    102,
    114,
    119,
    182,
    184,
    187,
    193,
    209,
    225,
    229,
    230,
    231,
    256,
    261,
    302,
    309,
    310,
    343,
    344,
    345,
    350,
    351,
    352,
    364,
    370,
    371,
    377,
    384,
    387,
    392,
    393,
    396,
    399,
    410,
    420,
    425,
    436,
    446,
    453,
    468,
    476,
    478,
    483,
    487,
    503,
    510,
    542,
    582,
    599,
    608,
    630,
    638,
    640,
    664,
    688,
    706,
    715,
    717,
    739,
    740,
    765,
    800,
    809,
    816,
    820,
    838,
    852,
    853,
    866,
    876,
    877,
    897,
    899,
    910,
    914,
    921,
    929,
    950,
    958,
    968,
    973,
    996,
    999,
    1004,
    1008,
    1016,
    1039,
    1046,
    1047,
    1050,
    1061,
    1079,
    1080,
    1088,
    1089,
    10

In [7]:
all_indices = []
for split in splits:
    for fold_name in ['train', 'val', 'test']:
        all_indices.extend(split.get(fold_name, []))

print("Max index in splits:", max(all_indices))


Max index in splits: 1112


In [8]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root="DATA/PROTEINS_full", name="PROTEINS")
print("Dataset size:", len(dataset))

Extracting DATA/PROTEINS_full/PROTEINS/PROTEINS.zip
Processing...


Dataset size: 1113


Done!
